In [1]:
import pandas as pd

## Class

In [89]:
from datetime import datetime,timedelta
from pandas import date_range,concat,DataFrame
from pickle import load
from numpy import ceil

class Predictions():
    def __init__(self,df):
        start=(pd.to_datetime(df["date"].max()) + timedelta(days=1)).strftime('%Y-%m-%d')
        end= ( datetime.strptime(start, '%Y-%m-%d') + timedelta(days=6)).strftime('%Y-%m-%d')
        date_index = date_range(start=start, end=end, freq="D")
        aux=DataFrame(columns=df.columns[1:], index=date_index)
        #aux.iloc[:,-12:-4]=df.iloc[-7:,-12:-4]
        self.df =concat([df.iloc[-50:,:],aux.reset_index().rename(columns={'index':'date'})],ignore_index=True)
        self.ls_predictions=[]
    
    def loadScaler(self,column):
        zone=column[8]
        crime=column[10:]
        scx_file="NewModels/sc_x_"+crime+"_"+zone+".pkl"
        scy_file="NewModels/sc_y_"+crime+"_"+zone+".pkl"
        with open(scx_file, "rb") as fp:
            sc= load(fp)
        with open(scy_file, "rb") as fp:
            sc2= load(fp)
        self.sc= sc
        self.sc2= sc2
    
    def loadLSTM(self,column):
        zone=column[8]
        crime=column[10:]
        with open("NewModels/XGBoost/modeloSem_"+zone+"_"+crime+".pkl", "rb") as m:
            model = load(m)
        self.model= model
        
    def scalerData(self,column,row):
        X,y=self.df.drop(["Target",'date'],axis=1).iloc[row,:].to_numpy(),self.df[["Target"]].iloc[row,:].to_numpy()

        X=self.sc.transform(X.reshape(1,-1))
        y=self.sc2.transform(y.reshape(-1,1))
        X= X.reshape(-1,X.shape[1])
        #X= X.reshape(X.shape[0], 1, X.shape[1])
        return X, y
    
    def forecast(self,X):
        #self.tomorrow=ceil(self.sc2.inverse_transform(self.model.predict(X)))[0][0]
        self.tomorrow= ceil(self.sc2.inverse_transform((self.model.predict(X)).reshape(-1,1)))[0][0]
        self.ls_predictions.append(self.tomorrow)
        
    def createMA(self,column):
        for i in range(10,50,10):
            self.df.iloc[:, int((i/10))-6]= self.df[column].rolling(window =i).mean()
            
    def generateNewDay(self,column,row):
        self.df.iloc[row, 1]= self.tomorrow
        self.df.iloc[row,2:35]=self.df.iloc[row-1,1:34]
        self.createMA(column)

## Test

In [90]:
df_feature= pd.read_csv("FeatureWE.csv")

In [91]:
predicter= Predictions(df_feature.iloc[:-7,:])

In [92]:
column="Crimes_Z4T100"

In [93]:
predicter.loadScaler(column)

In [94]:
predicter.loadLSTM(column)

In [95]:
for row in range(-8,0):
    X,y=predicter.scalerData(column,row)
    predicter.forecast(X)
    predicter.generateNewDay(column,row+1)

In [96]:
predicter.ls_predictions

[7.0, 8.0, 8.0, 7.0, 7.0, 7.0, 8.0, 8.0]

In [99]:
df_feature.iloc[-7:][["date","Crimes_Z4T100"]]

,date,Crimes_Z4T100
1107,2023-02-20,4.0
1108,2023-02-21,0.0
1109,2023-02-22,6.0
1110,2023-02-23,5.0
1111,2023-02-24,2.0
1112,2023-02-25,8.0
1113,2023-02-26,9.0


In [97]:
predicter.df.iloc[-7:,:][["date","Crimes_Z4T100"]]

,date,Crimes_Z4T100
50,2023-02-20 00:00:00,7.0
51,2023-02-21 00:00:00,8.0
52,2023-02-22 00:00:00,8.0
53,2023-02-23 00:00:00,7.0
54,2023-02-24 00:00:00,7.0
55,2023-02-25 00:00:00,7.0
56,2023-02-26 00:00:00,8.0
